<a href="https://colab.research.google.com/github/vitorsoyama/Titanic-Kaggle/blob/main/Titanic_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação de dados

In [ ]:
import pandas as pd
import random
import numpy as np

random.seed(123456)
np.random.seed(123456)

##Manipulação

In [ ]:
urlY_test = 'https://raw.githubusercontent.com/Ivysot/Titanic-Kaggle/main/Data/gender_submission.csv'
urlX_Y_train = 'https://raw.githubusercontent.com/Ivysot/Titanic-Kaggle/main/Data/train.csv'
urlX_test = 'https://raw.githubusercontent.com/Ivysot/Titanic-Kaggle/main/Data/test.csv'

y_test = pd.read_csv(urlY_test)
x_y_train = pd.read_csv(urlX_Y_train)
x_test = pd.read_csv(urlX_test)

print(y_test.head())
print(x_y_train.head())
print(x_test.head())

   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  S

In [ ]:
x_test = x_test.drop(['Name','Ticket','Cabin', 'Fare','Embarked' ], axis = 1)

gender = {
    'male': 0,
    'female': 1
}
x_test['Age'] = x_test['Age'].fillna('30.0')
x_test['Sex'] = x_test['Sex'].map(gender)
x_test = x_test.set_index('PassengerId')
x_y_train = x_y_train.set_index('PassengerId')
y_test = y_test.set_index('PassengerId')


In [ ]:
y_test = y_test.to_numpy()



In [ ]:
x_train = x_y_train.drop(['Survived','Name','Ticket','Cabin', 'Fare','Embarked' ], axis = 1)

gender = {
    'male': 0,
    'female': 1
}

x_train['Sex'] = x_train['Sex'].map(gender)
x_train['Age'] = x_train['Age'].fillna('30.0')
x_y_train['Sex'] = x_y_train['Sex'].map(gender)

y_train = x_y_train['Survived']




#Decision Tree Classifier

##Com a idade

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)


y_pred =  clf.predict(x_test)


In [ ]:
from sklearn.metrics import accuracy_score

print(f'A acurácia da Decision tree com a idade {accuracy_score(y_test, y_pred)}')

A acurácia da Decision tree com a idade 0.8181818181818182


##Sem idade

In [ ]:
from sklearn import tree
from sklearn.metrics import accuracy_score


x_train_no_age = x_train.drop('Age', axis = 1)
x_test_no_age = x_test.drop('Age', axis = 1)

clf_no_age = clf.fit(x_train_no_age, y_train)


y_pred_no_age =  clf_no_age.predict(x_test_no_age)

print(f'A acurácia da Decision tree {accuracy_score(y_test, y_pred_no_age)}')

A acurácia da Decision tree 0.9090909090909091


#KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x_train_no_age, y_train)

neigh_predict = neigh.predict(x_test_no_age)

print(f'A acurácia do KNN: {accuracy_score(y_test, neigh_predict)}')

A acurácia do KNN: 0.9186602870813397


#Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

NB = GaussianNB()
NB.fit(x_train_no_age, y_train)

NB_predict = NB.predict(x_test_no_age)

print(f'A acurácia do NB: {accuracy_score(y_test, NB_predict)}')

A acurácia do NB: 0.8564593301435407


#Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()
forest.fit(x_train_no_age, y_train)

forest_predict = forest.predict(x_test_no_age)

print(f'A acurácia do Random Forest: {accuracy_score(y_test, forest_predict)}')

A acurácia do Random Forest: 0.9043062200956937


#Linear SVC

In [ ]:
from sklearn.preprocessing import normalize

x_train_no_age_n = normalize(x_train_no_age)
x_test_no_age_n = normalize(x_test_no_age)

In [ ]:
from sklearn.svm import LinearSVC

linear = LinearSVC()
linear.fit(x_train_no_age_n, y_train)

linear_predict = linear.predict(x_test_no_age_n)

print(f'A acurácia do Linear SVC: {accuracy_score(y_test, linear_predict)}')

A acurácia do Linear SVC: 0.992822966507177


In [ ]:
linear.coef_

array([[0.07458347, 2.83095821, 0.04951119, 0.43595444]])

In [ ]:
x_test_scoef = x_test.drop(['Pclass', 'Age','SibSp','Parch'], axis = 1)
x_train_scoef = x_train.drop(['Pclass', 'Age','SibSp', 'Parch'], axis = 1)

linear = LinearSVC()
linear.fit(x_train_scoef, y_train)

linear_predict2 = linear.predict(x_test_scoef)

print(f'A acurácia do Linear SVC: {accuracy_score(y_test, linear_predict2)}')

A acurácia do Linear SVC: 1.0


#Dummy Classifier

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy = dummy_clf.fit(x_train_no_age, y_train)
dummy_predict = dummy.predict(x_test_no_age)
print(f'A acurácia da Dummy: {accuracy_score(y_test, dummy_predict)}')

A acurácia da Dummy: 0.6363636363636364


#Exportando dados

In [ ]:

y_pred_df = pd.DataFrame(linear_predict, index=x_test.index)
y_pred_df = y_pred_df.rename(columns={0:"Survived"})

y_pred_df.to_csv("LinearPred.csv",index = True, header = ["Survived"])

y_pred_df.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [ ]:
url = 'LinearPred.csv'

df = pd.read_csv(url)
df.head()



,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
